# LAB : Prompt Engineering pour les LLMs

### Installation des dépendances

In [ ]:
# Commandes bash à exécuter dans un terminal:
# # Créer un environnement virtuel (recommandé)
# python -m venv llm_lab_env
# source llm_lab_env/bin/activate  # Linux/Mac
# # llm_lab_env\Scripts\activate  # Windows
#
# # Installer les packages nécessaires
# pip install openai huggingface_hub tiktoken deepeval python-dotenv requests

In [1]:
!pip install openai huggingface_hub tiktoken deepeval python-dotenv requests

### Configuration des clés API

Créez un fichier `.env` à la racine de votre projet :

**Configuration `.env`:**
```
# ============================================
# CONFIGURATION DES CLÉS API
# ============================================

# Clé API OpenAI (obligatoire pour certains exercices)
OPENAI_API_KEY=sk-votre-cle-api-openai

# Clé API Hugging Face (gratuit - créer un compte sur huggingface.co)
HF_TOKEN=hf_votre-token-huggingface

# ============================================
# PARAMÈTRES PERSONNALISABLES
# ============================================

# Modèle OpenAI par défaut
DEFAULT_OPENAI_MODEL=gpt-4o-mini

# Modèle Hugging Face gratuit par défaut
DEFAULT_HF_MODEL=meta-llama/Meta-Llama-3-8B-Instruct
```

### Script d'initialisation

In [2]:
# config.py
import os
from dotenv import load_dotenv

load_dotenv()

class Config:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    HF_TOKEN = os.getenv("HF_TOKEN")
    DEFAULT_OPENAI_MODEL = os.getenv("DEFAULT_OPENAI_MODEL", "gpt-5-nano")
    DEFAULT_HF_MODEL = os.getenv("DEFAULT_HF_MODEL", "meta-llama/Meta-Llama-3-8B-Instruct")

config = Config()

---

## Partie 1 : Comprendre les Tokens et la Tokenisation


In [6]:
"""
Exercice 1 : Comprendre la tokenisation
Objectif : Visualiser comment le texte est découpé en tokens
"""

import tiktoken

def analyser_tokens(texte: str, modele: str = "gpt-5-nano"):
    """
    Analyse la tokenisation d'un texte pour un modèle donné.

    Args:
        texte: Le texte à analyser
        modele: Le modèle pour lequel tokeniser (gpt-4, gpt-3.5-turbo, etc.)
    """
    # Obtenir l'encodeur pour le modèle
    encodeur = tiktoken.encoding_for_model(modele)

    # Tokeniser le texte
    tokens = encodeur.encode(texte)

    # Afficher les résultats
    print(f"Texte original : '{texte}'")
    print(f"Nombre de tokens : {len(tokens)}")
    print(f"Ratio caractères/tokens : {len(texte)/len(tokens):.2f}")
    print(f"\n Détail des tokens :")

    for i, token_id in enumerate(tokens):
        token_texte = encodeur.decode([token_id])
        print(f"  Token {i+1}: ID={token_id:6d} → '{token_texte}'")

    return tokens

In [7]:
# Exercice 1.1 : Comparez la tokenisation de ces textes
textes_test = [
    "Hello, world!",
    "Bonjour le monde !",
    "L'intelligence artificielle transforme notre quotidien.",
    "The quick brown fox jumps over the lazy dog.",
    "12345 + 67890 = 80235",
    "🚀 Les émojis sont-ils bien tokenisés ? 🤖",
]

print("=" * 60)
print("ANALYSE DE TOKENISATION")
print("=" * 60)

for texte in textes_test:
    analyser_tokens(texte)
    print("-" * 60)

ANALYSE DE TOKENISATION
Texte original : 'Hello, world!'
Nombre de tokens : 4
Ratio caractères/tokens : 3.25

 Détail des tokens :
  Token 1: ID= 13225 → 'Hello'
  Token 2: ID=    11 → ','
  Token 3: ID=  2375 → ' world'
  Token 4: ID=     0 → '!'
------------------------------------------------------------
Texte original : 'Bonjour le monde !'
Nombre de tokens : 4
Ratio caractères/tokens : 4.50

 Détail des tokens :
  Token 1: ID= 45751 → 'Bonjour'
  Token 2: ID=   505 → ' le'
  Token 3: ID= 15807 → ' monde'
  Token 4: ID=  1073 → ' !'
------------------------------------------------------------
Texte original : 'L'intelligence artificielle transforme notre quotidien.'
Nombre de tokens : 9
Ratio caractères/tokens : 6.11

 Détail des tokens :
  Token 1: ID=    43 → 'L'
  Token 2: ID= 37062 → ''int'
  Token 3: ID= 33465 → 'elligence'
  Token 4: ID=105453 → ' artific'
  Token 5: ID= 22380 → 'ielle'
  Token 6: ID=184109 → ' transforme'
  Token 7: ID= 12092 → ' notre'
  Token 8: ID= 59486 

In [10]:
# Exercice 1.2 : Calculez le coût approximatif
# Prix GPT-4o-mini : ~$0.15 / 1M tokens input, ~$0.60 / 1M tokens output
def calculer_cout(nb_tokens_input: int, nb_tokens_output: int, modele: str = "gpt-5-nano"):
    """
    Calcule le coût approximatif d'une requête.

    TODO: Complétez cette fonction en utilisant les tarifs actuels
    Référence : https://platform.openai.com/docs/pricing
    """
    # 💡 Personnalisez les tarifs selon votre modèle
    tarifs = {
        "gpt-4o-mini": {"input": 0.15/1000000, "output": 0.60/1000000},
        "gpt-5-nano": {"input": 0.05/1000000, "output": 0.4/1000000},
        "gpt-4o": {"input": 2.50/1000000, "output": 10.00/1000000},
        # Ajoutez d'autres modèles ici
    }

    if modele not in tarifs:
        print(f"⚠️ Modèle {modele} non trouvé, utilisation de gpt-4o-mini")
        modele = "gpt-4o-mini"

    cout_input = nb_tokens_input * tarifs[modele]["input"]
    cout_output = nb_tokens_output * tarifs[modele]["output"]

    return {
        "input": cout_input,
        "output": cout_output,
        "total": cout_input + cout_output
    }

# Test du calcul de coût
print("\n💰 Estimation des coûts pour 1000 requêtes avec 500 tokens input et 200 tokens output:")
cout = calculer_cout(500 * 1000, 200 * 1000)
print(f"  Coût total estimé : ${cout['total']:.4f}")


💰 Estimation des coûts pour 1000 requêtes avec 500 tokens input et 200 tokens output:
  Coût total estimé : $0.1050


---

## Partie 2 : Configuration des Paramètres LLM

Les LLMs génèrent du texte en prédisant le token suivant basé sur une **distribution de probabilités**. Chaque token possible a une probabilité associée.

**Paramètres clés :**

| Paramètre | Plage | Effet |
|-----------|-------|-------|
| **Temperature** | 0.0 - 2.0 | Contrôle la "créativité" (randomness) |
| **Top-p** | 0.0 - 1.0 | Nucleus sampling - cumul des probabilités |
| **Top-k** | 1 - ∞ | Limite le nombre de tokens candidats |

### Pratique : Expérimenter avec les Paramètres

In [11]:
"""
Exercice 2 : Expérimenter avec les paramètres des LLMs
Objectif : Comprendre l'impact de temperature, top-p et top-k
"""

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generer_avec_parametres(
    prompt: str,
    temperature: float = 1.0,
    top_p: float = 1.0,
    max_tokens: int = 100,
    n_generations: int = 3,
    model: str = "gpt-4o-mini"
):
    """
    Génère plusieurs réponses avec les mêmes paramètres pour observer la variabilité.

    Args:
        prompt: Le prompt à envoyer au modèle
        temperature: Contrôle la randomness (0=déterministe, 2=très créatif)
        top_p: Nucleus sampling (0.1=très focalisé, 1.0=tous les tokens)
        max_tokens: Nombre maximum de tokens à générer
        n_generations: Nombre de générations à effectuer
        model: Modèle à utiliser
    """
    print(f"\n{'='*60}")
    print(f"Paramètres : temperature={temperature}, top_p={top_p}")
    print(f"Prompt : {prompt[:50]}...")
    print("="*60)

    reponses = []
    for i in range(n_generations):
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )
        reponse = response.choices[0].message.content
        reponses.append(reponse)
        print(f"\nGénération {i+1}:\n{reponse}")

    # Analyse de la variabilité
    mots_uniques = set()
    for r in reponses:
        mots_uniques.update(r.lower().split())

    print(f"\nAnalyse : {len(mots_uniques)} mots uniques sur {n_generations} générations")

    return reponses

In [12]:
# Exercice 2.1 : Comparez les effets de différentes températures
prompt_creatif = "Écrivez le début d'une histoire fantastique en une phrase."

print("\n" + "*" * 20)
print("EXPÉRIENCE 1 : EFFET DE LA TEMPÉRATURE")
print("*" * 20)

# TODO: Testez avec temperature = 0.0, 0.5, 1.0, 1.5
configurations_temp = [
    {"temperature": 0.0, "top_p": 1.0},  # Déterministe
    {"temperature": 0.5, "top_p": 1.0},  # Légèrement créatif
    {"temperature": 1.0, "top_p": 1.0},  # Équilibré
    {"temperature": 1.5, "top_p": 1.0},  # Très créatif
]

for config in configurations_temp:
    generer_avec_parametres(
        prompt_creatif,
        temperature=config["temperature"],
        top_p=config["top_p"],
        n_generations=3
    )


********************
EXPÉRIENCE 1 : EFFET DE LA TEMPÉRATURE
********************

Paramètres : temperature=0.0, top_p=1.0
Prompt : Écrivez le début d'une histoire fantastique en une...

Génération 1:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Génération 2:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Génération 3:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Analyse : 32 mots uniques sur 3 générations

Paramètres : tempe

In [13]:
# Exercice 2.2 : Comparez les effets de différents top_p
print("\n" + "*" * 20)
print("EXPÉRIENCE 2 : EFFET DU TOP-P")
print("*" * 20)

# TODO: Testez avec top_p = 0.1, 0.5, 0.9, 1.0
configurations_top_p = [
    {"temperature": 1.0, "top_p": 0.1},  # Très focalisé
    {"temperature": 1.0, "top_p": 0.5},  # Moyennement focalisé
    {"temperature": 1.0, "top_p": 0.9},  # Diversifié
    {"temperature": 1.0, "top_p": 1.0},  # Maximum de diversité
]

for config in configurations_top_p:
    generer_avec_parametres(
        prompt_creatif,
        temperature=config["temperature"],
        top_p=config["top_p"],
        n_generations=3
    )


********************
EXPÉRIENCE 2 : EFFET DU TOP-P
********************

Paramètres : temperature=1.0, top_p=0.1
Prompt : Écrivez le début d'une histoire fantastique en une...

Génération 1:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Génération 2:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Génération 3:
Dans un royaume oublié où les étoiles murmuraient des secrets anciens, une jeune fille découvrit une clé en argent scintillante enfouie sous les racines d'un arbre millénaire, promettant d'ouvrir la porte vers des mondes insoupçonnés.

Analyse : 32 mots uniques sur 3 générations

Paramètres : temperature=1.

In [14]:
# Exercice 2.3 : Cas d'usage pratiques
print("\n" + "*" * 20)
print("EXPÉRIENCE 3 : CAS D'USAGE PRATIQUES")
print("*" * 20)

cas_usage = {
    "extraction_donnees": {
        "prompt": "Extrayez les informations suivantes du texte : nom, date, montant.\nTexte: 'Le contrat signé par Jean Dupont le 15 mars 2024 porte sur un montant de 50000 euros.'",
        "temperature": 0.0,  # Déterministe pour l'extraction
        "top_p": 1.0
    },
    "redaction_creative": {
        "prompt": "Proposez un slogan publicitaire original pour une marque de café éco-responsable.",
        "temperature": 1.2,  # Créatif pour la pub
        "top_p": 0.9
    },
    "code_generation": {
        "prompt": "Écrivez une fonction Python qui calcule la suite de Fibonacci.",
        "temperature": 0.2,  # Précis pour le code
        "top_p": 0.95
    },
    "conversation": {
        "prompt": "Que pensez-vous de l'intelligence artificielle ?",
        "temperature": 0.7,  # Équilibré pour la conversation
        "top_p": 0.9
    }
}

# 💡 Personnalisez les paramètres selon vos observations
for nom, config in cas_usage.items():
    print(f"\nCas d'usage : {nom}")
    generer_avec_parametres(
        config["prompt"],
        temperature=config["temperature"],
        top_p=config["top_p"],
        n_generations=2
    )


********************
EXPÉRIENCE 3 : CAS D'USAGE PRATIQUES
********************

Cas d'usage : extraction_donnees

Paramètres : temperature=0.0, top_p=1.0
Prompt : Extrayez les informations suivantes du texte : nom...

Génération 1:
Nom : Jean Dupont  
Date : 15 mars 2024  
Montant : 50000 euros

Génération 2:
Nom : Jean Dupont  
Date : 15 mars 2024  
Montant : 50000 euros

Analyse : 11 mots uniques sur 2 générations

Cas d'usage : redaction_creative

Paramètres : temperature=1.2, top_p=0.9
Prompt : Proposez un slogan publicitaire original pour une ...

Génération 1:
"Réveillez vos sens, préservez la planète : Savourez l'éthique, goûtez l'engagement."

Génération 2:
"Réveillez vos sens, préservez la planète : chaque tasse compte !"

Analyse : 15 mots uniques sur 2 générations

Cas d'usage : code_generation

Paramètres : temperature=0.2, top_p=0.95
Prompt : Écrivez une fonction Python qui calcule la suite d...

Génération 1:
Voici une fonction Python qui calcule la suite de Fibonacci. L

---

## Partie 3 : Les Composantes d'un Prompt Efficace

### Structure d'un Prompt

Un prompt efficace contient généralement à minima :

```
┌─────────────────────────────────────────┐
│  1. CONTEXTE / RÔLE                     │
│     "Tu es un expert en..."             │
├─────────────────────────────────────────┤
│  2. INSTRUCTION                         │
│     "Analyse le texte suivant..."       │
├─────────────────────────────────────────┤
│  3. DONNÉES D'ENTRÉE                    │
│     [Le contenu à traiter]              │
├─────────────────────────────────────────┤
│  4. FORMAT DE SORTIE                    │
│     "Réponds en JSON avec..."           │
└─────────────────────────────────────────┘
```

### Exercice Pratique : Construire des Prompts Structurés

In [18]:
"""
Exercice 3 : Construction de prompts structurés
Objectif : Maîtriser les composantes d'un prompt efficace
"""

from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class PromptBuilder:
    """
    Classe pour construire des prompts structurés de manière modulaire.
    """

    def __init__(self):
        self.contexte = ""
        self.instruction = ""
        self.donnees = ""
        self.format_sortie = ""
        self.exemples = []

    def definir_contexte(self, role: str, expertise: str = "", contraintes: str = ""):
        """Définit le contexte et le rôle de l'assistant."""
        self.contexte = f"Tu es {role}."
        if expertise:
            self.contexte += f" Tu possèdes une expertise en {expertise}."
        if contraintes:
            self.contexte += f" {contraintes}"
        return self

    def definir_instruction(self, action: str, details: str = ""):
        """Définit l'instruction principale."""
        self.instruction = action
        if details:
            self.instruction += f" {details}"
        return self

    def ajouter_donnees(self, donnees: str, label: str = "Données"):
        """Ajoute les données d'entrée."""
        self.donnees = f"\n\n### {label}\n{donnees}"
        return self

    def definir_format(self, format_type: str, structure: dict = None):
        """Définit le format de sortie attendu."""
        self.format_sortie = f"\n\n### Format de réponse\nRéponds au format {format_type}."
        if structure:
            self.format_sortie += f"\nStructure attendue:\n```json\n{json.dumps(structure, indent=2, ensure_ascii=False)}\n```"
        return self

    def ajouter_exemple(self, entree: str, sortie: str):
        """Ajoute un exemple (few-shot)."""
        self.exemples.append({"entree": entree, "sortie": sortie})
        return self

    def construire(self) -> str:
        """Construit le prompt final."""
        prompt = self.contexte
        prompt += f"\n\n### Instruction\n{self.instruction}"

        if self.exemples:
            prompt += "\n\n### Exemples"
            for i, ex in enumerate(self.exemples, 1):
                prompt += f"\n\n**Exemple {i}:**\nEntrée: {ex['entree']}\nSortie: {ex['sortie']}"

        prompt += self.donnees
        prompt += self.format_sortie

        return prompt

def executer_prompt(prompt: str, model: str = "gpt-4.1-mini", temperature: float = 0.7):
    """Exécute un prompt et retourne la réponse."""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

In [19]:
# Exercice 3.1 : Construire un prompt pour l'analyse de sentiment
print("\nEXERCICE 3.1 : Analyse de Sentiment")
print("="*60)

# 💡 Personnalisez ce texte avec vos propres données
texte_analyse = """
Le nouveau restaurant du quartier est une vraie déception.
L'attente était interminable malgré une réservation,
et les plats, bien que joliment présentés, manquaient cruellement de saveur.
Cependant, le service était impeccable et le cadre vraiment agréable.
"""

prompt_sentiment = (
    PromptBuilder()
    .definir_contexte(
        role="un analyste de sentiment expert",
        expertise="l'analyse des avis clients",
        contraintes="Tu dois être précis et nuancé dans ton analyse."
    )
    .definir_instruction(
        action="Analyse le sentiment du texte suivant.",
        details="Identifie les aspects positifs et négatifs mentionnés."
    )
    .ajouter_donnees(texte_analyse, label="Avis client")
    .definir_format(
        format_type="JSON",
        structure={
            "sentiment_global": "positif|négatif|mixte",
            "score": "0-10",
            "aspects_positifs": ["liste des points positifs"],
            "aspects_negatifs": ["liste des points négatifs"],
            "resume": "résumé en une phrase"
        }
    )
    .construire()
)

print("Prompt construit:")
print("-"*40)
print(prompt_sentiment)
print("-"*40)

reponse = executer_prompt(prompt_sentiment, temperature=0.3)
print("\nRéponse du modèle:")
print(reponse)


EXERCICE 3.1 : Analyse de Sentiment
Prompt construit:
----------------------------------------
Tu es un analyste de sentiment expert. Tu possèdes une expertise en l'analyse des avis clients. Tu dois être précis et nuancé dans ton analyse.

### Instruction
Analyse le sentiment du texte suivant. Identifie les aspects positifs et négatifs mentionnés.

### Avis client

Le nouveau restaurant du quartier est une vraie déception. 
L'attente était interminable malgré une réservation, 
et les plats, bien que joliment présentés, manquaient cruellement de saveur.
Cependant, le service était impeccable et le cadre vraiment agréable.


### Format de réponse
Réponds au format JSON.
Structure attendue:
```json
{
  "sentiment_global": "positif|négatif|mixte",
  "score": "0-10",
  "aspects_positifs": [
    "liste des points positifs"
  ],
  "aspects_negatifs": [
    "liste des points négatifs"
  ],
  "resume": "résumé en une phrase"
}
```
----------------------------------------

Réponse du modèle:
``

In [20]:
# Exercice 3.2 : Extraction d'entités nommées
print("\n\nEXERCICE 3.2 : Extraction d'Entités Nommées")
print("="*60)

# 💡 Personnalisez ce texte
texte_entites = """
Apple Inc., dirigée par Tim Cook, a annoncé lors de la conférence WWDC 2024
à San José le lancement de nouveaux produits. L'action AAPL a gagné 5%
suite à cette annonce, atteignant 195 dollars. Le département R&D,
basé à Cupertino en Californie, a investi 2,5 milliards de dollars
dans le développement de l'IA cette année.
"""

# TODO: Complétez la construction du prompt
prompt_entites = (
    PromptBuilder()
    .definir_contexte(
        role="un spécialiste en extraction d'information",
        # Ajoutez expertise et contraintes
    )
    .definir_instruction(
        action="Extrais toutes les entités nommées du texte.",
        # Ajoutez des détails
    )
    .ajouter_donnees(texte_entites, label="Texte à analyser")
    .definir_format(
        format_type="JSON",
        structure={
            "personnes": [],
            "organisations": [],
            "lieux": [],
            "dates": [],
            "montants": [],
            "pourcentages": []
        }
    )
    .construire()
)

reponse_entites = executer_prompt(prompt_entites, temperature=0.1)
print("Entités extraites:")
print(reponse_entites)



EXERCICE 3.2 : Extraction d'Entités Nommées
Entités extraites:
```json
{
  "personnes": ["Tim Cook"],
  "organisations": ["Apple Inc.", "AAPL", "R&D"],
  "lieux": ["San José", "Cupertino", "Californie"],
  "dates": ["WWDC 2024", "cette année"],
  "montants": ["195 dollars", "2,5 milliards de dollars"],
  "pourcentages": ["5%"]
}
```


## Partie 4 : Few-Shot Learning

### Zero-Shot, One-Shot et Few-Shot

| Approche | Description | Utilisation |
|----------|-------------|-------------|
| **Zero-Shot** | Aucun exemple fourni | Tâches simples et communes |
| **One-Shot** | Un seul exemple | Tâches avec format spécifique |
| **Few-Shot** | Plusieurs exemples (2-5) | Tâches complexes ou inhabituelles |

### Pratique : Comparaison des Approches

In [21]:
"""
Exercice 4 : Few-Shot Learning
Objectif : Comprendre l'impact des exemples sur la qualité des réponses
"""

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def few_shot_classification(texte: str, exemples: list = None, model: str = "gpt-4o-mini"):
    """
    Effectue une classification de sentiment avec différents niveaux de few-shot.

    Args:
        texte: Le texte à classifier
        exemples: Liste de tuples (texte, label) pour le few-shot
        model: Modèle à utiliser
    """
    # Construction du prompt
    system_prompt = """Tu es un classificateur de sentiment.
    Classe chaque texte comme : POSITIF, NÉGATIF ou NEUTRE.
    Réponds UNIQUEMENT avec l'un de ces trois mots."""

    user_prompt = ""

    # Ajout des exemples few-shot
    if exemples:
        user_prompt += "Voici des exemples de classification :\n\n"
        for ex_texte, ex_label in exemples:
            user_prompt += f"Texte: \"{ex_texte}\"\nClassification: {ex_label}\n\n"
        user_prompt += "---\n\n"

    user_prompt += f"Texte à classifier: \"{texte}\"\nClassification:"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
        max_tokens=10
    )

    return response.choices[0].message.content.strip()

In [23]:
# Textes de test
textes_test = [
    "Ce produit a complètement changé ma vie quotidienne, je suis ravi !",
    "Le colis est arrivé à la date prévue.",
    "Franchement décevant, je m'attendais à beaucoup mieux pour ce prix.",
    "Le design est correct mais la qualité laisse à désirer.",
    "Je ne sais pas trop quoi penser de cet achat.",
]

# Exemples pour le few-shot
# Personnalisez ces exemples selon votre domaine
exemples_few_shot = [
    ("Excellent produit, livraison rapide, je recommande vivement !", "POSITIF"),
    ("Produit défectueux, remboursement difficile, une catastrophe.", "NÉGATIF"),
    ("Le produit correspond à la description.", "NEUTRE"),
    ("J'adore cette marque, c'est toujours de la qualité !", "POSITIF"),
    ("Déçu par la qualité, ne correspond pas aux photos.", "NÉGATIF"),
]

In [24]:
# Exercice 4.1 : Comparaison Zero-Shot vs Few-Shot
print("="*70)
print("COMPARAISON : ZERO-SHOT vs ONE-SHOT vs FEW-SHOT")
print("="*70)

resultats = {
    "zero_shot": [],
    "one_shot": [],
    "three_shot": [],
    "five_shot": []
}

for texte in textes_test:
    print(f"\nTexte: \"{texte[:50]}...\"")

    # Zero-Shot
    res_zero = few_shot_classification(texte, exemples=None)
    resultats["zero_shot"].append(res_zero)
    print(f"  Zero-Shot  : {res_zero}")

    # One-Shot
    res_one = few_shot_classification(texte, exemples=exemples_few_shot[:1])
    resultats["one_shot"].append(res_one)
    print(f"  One-Shot   : {res_one}")

    # Three-Shot
    res_three = few_shot_classification(texte, exemples=exemples_few_shot[:3])
    resultats["three_shot"].append(res_three)
    print(f"  Three-Shot : {res_three}")

    # Five-Shot
    res_five = few_shot_classification(texte, exemples=exemples_few_shot)
    resultats["five_shot"].append(res_five)
    print(f"  Five-Shot  : {res_five}")

COMPARAISON : ZERO-SHOT vs ONE-SHOT vs FEW-SHOT

Texte: "Ce produit a complètement changé ma vie quotidienn..."
  Zero-Shot  : POSITIF
  One-Shot   : POSITIF
  Three-Shot : POSITIF
  Five-Shot  : POSITIF

Texte: "Le colis est arrivé à la date prévue...."
  Zero-Shot  : POSITIF
  One-Shot   : NEUTRE
  Three-Shot : NEUTRE
  Five-Shot  : NEUTRE

Texte: "Franchement décevant, je m'attendais à beaucoup mi..."
  Zero-Shot  : NÉGATIF
  One-Shot   : NÉGATIF
  Three-Shot : NÉGATIF
  Five-Shot  : NÉGATIF

Texte: "Le design est correct mais la qualité laisse à dés..."
  Zero-Shot  : NÉGATIF
  One-Shot   : NÉGATIF
  Three-Shot : NÉGATIF
  Five-Shot  : NÉGATIF

Texte: "Je ne sais pas trop quoi penser de cet achat...."
  Zero-Shot  : NEUTRE
  One-Shot   : NEUTRE
  Three-Shot : NEUTRE
  Five-Shot  : NEUTRE


In [25]:


# Exercice 4.2 : Few-Shot pour la génération de format spécifique
print("\n\n" + "="*70)
print("FEW-SHOT POUR GÉNÉRATION DE FORMAT SPÉCIFIQUE")
print("="*70)

def generer_fiche_produit(nom_produit: str, description: str, exemples: list = None):
    """
    Génère une fiche produit structurée en utilisant le few-shot learning.
    """
    system_prompt = "Tu es un rédacteur marketing expert."

    user_prompt = "Génère une fiche produit structurée.\n\n"

    if exemples:
        user_prompt += "Exemples de fiches produits :\n\n"
        for ex in exemples:
            user_prompt += f"---\n{ex}\n---\n\n"

    user_prompt += f"""
Produit: {nom_produit}
Description brute: {description}

Génère la fiche produit:"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content

# Exemples de fiches produits pour le few-shot
# 💡 Personnalisez selon votre secteur d'activité
exemples_fiches = [
    """
**Nom:** Casque Audio ProSound X1
**Tagline:** L'excellence sonore à portée d'oreilles
**Points clés:**
• Réduction de bruit active -40dB
• Autonomie 30 heures
• Bluetooth 5.3
**Prix:** 149,99€
**Note:** ⭐⭐⭐⭐⭐ (4.8/5)
""",
    """
**Nom:** Montre Connectée FitPulse
**Tagline:** Votre santé au poignet
**Points clés:**
• Suivi cardiaque 24/7
• GPS intégré
• Étanche 50m
**Prix:** 199,99€
**Note:** ⭐⭐⭐⭐ (4.5/5)
"""
]

# Test avec et sans exemples
produit_test = {
    "nom": "Lampe de Bureau SmartLight",
    "description": "lampe LED avec contrôle tactile, 5 niveaux de luminosité, port USB, température de couleur ajustable 2700K-6500K, consommation 12W"
}

print("\nSans exemples (Zero-Shot):")
print(generer_fiche_produit(produit_test["nom"], produit_test["description"]))

print("\nAvec exemples (Few-Shot):")
print(generer_fiche_produit(produit_test["nom"], produit_test["description"], exemples_fiches))



FEW-SHOT POUR GÉNÉRATION DE FORMAT SPÉCIFIQUE

Sans exemples (Zero-Shot):
### Fiche Produit : Lampe de Bureau SmartLight

---

**Nom du produit :** Lampe de Bureau SmartLight

**Catégorie :** Éclairage / Accessoires de Bureau

**Description :**
La Lampe de Bureau SmartLight allie modernité et fonctionnalité pour illuminer votre espace de travail. Grâce à sa technologie LED avancée, elle offre une expérience d’éclairage personnalisée, parfaite pour répondre à tous vos besoins.

---

#### Caractéristiques Principales :

- **Type d'éclairage :** LED
- **Contrôle tactile :** Oui, pour un ajustement facile
- **Niveaux de luminosité :** 5 niveaux (de doux à intense)
- **Température de couleur :** Ajustable de 2700K (lumière chaude) à 6500K (lumière froide)
- **Consommation d'énergie :** 12W
- **Port USB :** Oui, pour charger vos appareils électroniques

---

#### Avantages :

- **Personnalisation de l'éclairage :** Adaptez la lumière selon vos activités (lecture, travail, détente) grâce à 

---

## Partie 5 : Chain of Thought (CoT)

### Raisonnement Étape par Étape

Le **Chain of Thought** encourage le modèle à expliciter son raisonnement avant de donner une réponse finale. Cette technique améliore significativement les performances sur les tâches de raisonnement complexe.

**Variantes :**
- **Zero-Shot CoT** : Ajouter "Réfléchissons étape par étape" au prompt
- **Few-Shot CoT** : Fournir des exemples avec raisonnement détaillé
- **Self-Consistency** : Générer plusieurs raisonnements et voter

### Pratique : Implémentation du CoT

In [26]:
"""
Exercice 5 : Chain of Thought Prompting
Objectif : Améliorer les capacités de raisonnement du LLM
"""

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def resoudre_probleme(probleme: str, methode: str = "direct", model: str = "gpt-4o-mini"):
    """
    Résout un problème avec différentes méthodes de prompting.

    Args:
        probleme: Le problème à résoudre
        methode: "direct", "zero_shot_cot", "few_shot_cot"
        model: Modèle à utiliser
    """

    if methode == "direct":
        # Réponse directe sans raisonnement
        prompt = f"{probleme}\n\nRéponds directement avec la réponse."

    elif methode == "zero_shot_cot":
        # Zero-shot Chain of Thought
        prompt = f"""{probleme}

        Réfléchissons étape par étape.
        1. Identifie les informations clés
        2. Établis les relations entre elles
        3. Effectue les calculs nécessaires
        4. Vérifie ta réponse

        Raisonnement:"""

    elif methode == "few_shot_cot":
        # Few-shot Chain of Thought avec exemples
        prompt = f"""Voici comment résoudre des problèmes de logique étape par étape :

        **Exemple 1:**
        Problème: Marie a 3 pommes. Elle en donne la moitié à Paul, puis achète 4 pommes. Combien a-t-elle de pommes ?

        Raisonnement:
        1. Marie commence avec 3 pommes
        2. Elle donne la moitié à Paul : 3 ÷ 2 = 1.5, mais on ne peut pas couper une pomme, donc elle donne 1 pomme (ou 2 si on arrondit au supérieur). Supposons qu'elle donne 1 pomme.
        3. Après avoir donné : 3 - 1 = 2 pommes
        4. Elle achète 4 pommes : 2 + 4 = 6 pommes

        Réponse: Marie a 6 pommes.

        **Exemple 2:**
        Problème: Un train part de Paris à 8h et arrive à Lyon à 10h. Un autre train part de Lyon à 9h et arrive à Paris à 11h. À quelle heure les trains se croisent-ils ?

        Raisonnement:
        1. Train 1 : Paris→Lyon, départ 8h, arrivée 10h, durée = 2h
        2. Train 2 : Lyon→Paris, départ 9h, arrivée 11h, durée = 2h
        3. Distance = D. Vitesse train 1 = D/2. Vitesse train 2 = D/2.
        4. À 9h, train 1 a parcouru 1h, donc D/2 de distance.
        5. Train 1 est à mi-chemin, train 2 part.
        6. Ils se rapprochent à vitesse D/2 + D/2 = D par heure.
        7. Distance restante = D/2. Temps = (D/2) / D = 0.5h = 30 min.
        8. Croisement : 9h + 30min = 9h30

        Réponse: Les trains se croisent à 9h30.

        ---

        **Nouveau problème:**
        {probleme}

        Raisonnement:"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1000
    )

    return response.choices[0].message.content

In [27]:
# Problèmes de test
# Ajoutez vos propres problèmes
problemes = [
    """
    Dans une entreprise de 120 employés, 40% travaillent au département marketing,
    25% au département technique, et le reste au département administratif.
    Le département marketing a reçu une prime de 500€ par employé.
    Le département technique a reçu une prime de 750€ par employé.
    Le département administratif a reçu une prime de 400€ par employé.
    Quel est le montant total des primes distribuées ?
    """,
    #"""
    #Alice, Bob et Charlie participent à une course. Alice termine avant Bob.
    #Charlie ne termine pas dernier. Bob termine après Charlie.
    #Dans quel ordre ont-ils terminé la course ?
    #""",
    #"""
    #Un fermier a des poules et des lapins dans sa ferme.
    #Il compte 20 têtes et 56 pattes au total.
    #Combien a-t-il de poules et combien a-t-il de lapins ?
    #"""
]

print("="*70)
print("COMPARAISON DES MÉTHODES DE RAISONNEMENT")
print("="*70)

for i, probleme in enumerate(problemes, 1):
    print(f"\n{'='*70}")
    print(f"PROBLÈME {i}")
    print(f"{'='*70}")
    print(probleme.strip())

    print("\n" + "-"*35)
    print("Méthode DIRECTE:")
    print("-"*35)
    print(resoudre_probleme(probleme, methode="direct"))

    print("\n" + "-"*35)
    print("Méthode ZERO-SHOT CoT:")
    print("-"*35)
    print(resoudre_probleme(probleme, methode="zero_shot_cot"))

    print("\n" + "-"*35)
    print("Méthode FEW-SHOT CoT:")
    print("-"*35)
    print(resoudre_probleme(probleme, methode="few_shot_cot"))

COMPARAISON DES MÉTHODES DE RAISONNEMENT

PROBLÈME 1
Dans une entreprise de 120 employés, 40% travaillent au département marketing, 
    25% au département technique, et le reste au département administratif.
    Le département marketing a reçu une prime de 500€ par employé.
    Le département technique a reçu une prime de 750€ par employé.
    Le département administratif a reçu une prime de 400€ par employé.
    Quel est le montant total des primes distribuées ?

-----------------------------------
Méthode DIRECTE:
-----------------------------------
Le montant total des primes distribuées est de 62 500 €.

-----------------------------------
Méthode ZERO-SHOT CoT:
-----------------------------------
Pour résoudre le problème, suivons les étapes que vous avez indiquées.

### 1. Identifie les informations clés
- Total des employés : 120
- Pourcentage d'employés au département marketing : 40%
- Pourcentage d'employés au département technique : 25%
- Pourcentage d'employés au départemen

---

## Partie 6 : ReAct Prompting

### Raisonnement et Action

**ReAct** (Reasoning + Acting) combine le raisonnement explicite avec l'exécution d'actions. Le modèle alterne entre :
- **Thought** (Pensée) : Réflexion sur la situation
- **Action** : Exécution d'une action (recherche, calcul, etc.)
- **Observation** : Résultat de l'action

### Pratique : Implémentation de ReAct

In [28]:
"""
Exercice 6 : ReAct Prompting
Objectif : Combiner raisonnement et actions pour des tâches complexes
"""

from openai import OpenAI
import os
import re
import json
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class ReactAgent:
    """
    Agent ReAct simplifié avec des outils simulés.
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model
        self.historique = []
        self.outils = {
            "recherche": self._recherche,
            "calcul": self._calcul,
            "date": self._date,
        }

    def _recherche(self, query: str) -> str:
        """Simule une recherche (à remplacer par une vraie API)."""
        # Personnalisez avec une vraie API de recherche
        base_connaissances = {
            "capitale france": "La capitale de la France est Paris.",
            "population paris": "Paris compte environ 2,1 millions d'habitants (ville) et 12 millions (agglomération).",
            "tour eiffel hauteur": "La Tour Eiffel mesure 330 mètres avec son antenne.",
            "python créateur": "Python a été créé par Guido van Rossum en 1991.",
            "openai fondation": "OpenAI a été fondée en 2015 par Sam Altman, Elon Musk et d'autres.",
        }

        query_lower = query.lower()
        for cle, valeur in base_connaissances.items():
            if any(mot in query_lower for mot in cle.split()):
                return valeur

        return f"Aucune information trouvée pour: {query}"

    def _calcul(self, expression: str) -> str:
        """Effectue un calcul mathématique."""
        try:
            # Nettoyage de l'expression
            expression_clean = expression.replace("^", "**").replace("×", "*").replace("÷", "/")
            resultat = eval(expression_clean)
            return f"Résultat: {resultat}"
        except Exception as e:
            return f"Erreur de calcul: {e}"

    def _date(self, query: str) -> str:
        """Retourne la date actuelle."""
        from datetime import datetime
        return f"Date actuelle: {datetime.now().strftime('%d/%m/%Y %H:%M')}"

    def executer_action(self, action: str, parametre: str) -> str:
        """Exécute une action et retourne l'observation."""
        action = action.lower().strip()
        if action in self.outils:
            return self.outils[action](parametre)
        return f"Action inconnue: {action}. Actions disponibles: {list(self.outils.keys())}"

    def resoudre(self, question: str, max_iterations: int = 5) -> str:
        """
        Résout une question en utilisant le pattern ReAct.
        """
        system_prompt = """Tu es un assistant qui résout des problèmes en utilisant le pattern ReAct.

        À chaque étape, tu dois:
        1. THOUGHT: Réfléchir à ce que tu sais et ce que tu dois faire
        2. ACTION: Choisir une action parmi [recherche, calcul, date]
        3. Attendre l'OBSERVATION

        Format de réponse pour chaque étape:
        THOUGHT: [ta réflexion]
        ACTION: [nom_action]
        INPUT: [paramètre de l'action]

        Quand tu as la réponse finale, écris:
        THOUGHT: [réflexion finale]
        FINAL_ANSWER: [ta réponse complète]

        Actions disponibles:
        - recherche: Recherche d'information (INPUT: mots-clés)
        - calcul: Calcul mathématique (INPUT: expression mathématique)
        - date: Obtenir la date actuelle (INPUT: vide)"""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Question: {question}"}
        ]

        print(f"\n{'='*60}")
        print(f"Question: {question}")
        print("="*60)

        for iteration in range(max_iterations):
            print(f"\n--- Itération {iteration + 1} ---")

            # Obtenir la prochaine étape du modèle
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.3,
                max_tokens=500
            )

            reponse_texte = response.choices[0].message.content
            print(f"\nModèle:\n{reponse_texte}")

            # Vérifier si on a une réponse finale
            if "FINAL_ANSWER:" in reponse_texte:
                final = reponse_texte.split("FINAL_ANSWER:")[-1].strip()
                print(f"\nRéponse finale: {final}")
                return final

            # Parser l'action
            action_match = re.search(r"ACTION:\s*(\w+)", reponse_texte)
            input_match = re.search(r"INPUT:\s*(.+?)(?:\n|$)", reponse_texte)

            if action_match:
                action = action_match.group(1)
                parametre = input_match.group(1) if input_match else ""

                # Exécuter l'action
                observation = self.executer_action(action, parametre)
                print(f"\nObservation: {observation}")

                # Ajouter au contexte
                messages.append({"role": "assistant", "content": reponse_texte})
                messages.append({"role": "user", "content": f"OBSERVATION: {observation}"})
            else:
                print("Aucune action détectée, ajout au contexte...")
                messages.append({"role": "assistant", "content": reponse_texte})

        return "Nombre maximum d'itérations atteint."

In [29]:
# Création de l'agent
agent = ReactAgent()

In [30]:
# Exercice 6.1 : Questions nécessitant recherche + raisonnement
print("\n" + "*"*20)
print("EXERCICE 6.1 : QUESTIONS COMPLEXES")
print("*"*20)

questions = [
    "Quelle est la population de la capitale de la France ?",
    "Si la Tour Eiffel mesurait 2 fois sa hauteur actuelle, combien mesurerait-elle ?",
    #"En quelle année Python a-t-il été créé, et quel âge a ce langage aujourd'hui ?",
]

for q in questions:
    agent.resoudre(q)
    print("\n" + "-"*60)


********************
EXERCICE 6.1 : QUESTIONS COMPLEXES
********************

Question: Quelle est la population de la capitale de la France ?

--- Itération 1 ---

Modèle:
THOUGHT: Je sais que la capitale de la France est Paris, et je dois rechercher la population actuelle de cette ville. 
ACTION: recherche
INPUT: population Paris 2023

Observation: Paris compte environ 2,1 millions d'habitants (ville) et 12 millions (agglomération).

--- Itération 2 ---

Modèle:
THOUGHT: J'ai trouvé que Paris a une population d'environ 2,1 millions d'habitants pour la ville elle-même, et environ 12 millions pour l'agglomération. Je vais maintenant formuler ma réponse finale. 
FINAL_ANSWER: La population de la capitale de la France, Paris, est d'environ 2,1 millions d'habitants, tandis que l'agglomération parisienne compte environ 12 millions d'habitants.

Réponse finale: La population de la capitale de la France, Paris, est d'environ 2,1 millions d'habitants, tandis que l'agglomération parisienne co

---

## Partie 7 : Évaluation des LLMs

### Métriques d'Évaluation

| Métrique | Description | Utilisation |
|----------|-------------|-------------|
| **Exactitude** | Correspondance exacte | Classification, QA fermé |
| **BLEU/ROUGE** | Similarité de texte | Traduction, résumé |
| **Coherence** | Logique du discours | Génération longue |
| **Faithfulness** | Fidélité aux sources | RAG, résumé |
| **G-Eval** | Évaluation par LLM | Tout type de tâche |

### Pratique : Évaluation avec DeepEval

In [38]:
"""
Exercice 7 : Évaluation des LLMs
Objectif : Mesurer et améliorer la qualité des réponses
"""

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ============================================
# 7.1 ÉVALUATION MANUELLE STRUCTURÉE
# ============================================

class EvaluateurManuel:
    """
    Évaluateur manuel avec critères structurés.
    """

    def __init__(self, criteres: dict = None):
        self.criteres = criteres or {
            "pertinence": "La réponse répond-elle à la question posée ?",
            "exactitude": "Les informations sont-elles correctes ?",
            "completude": "La réponse est-elle complète ?",
            "clarte": "La réponse est-elle claire et bien structurée ?",
            "concision": "La réponse est-elle concise sans être trop courte ?",
        }

    def evaluer_avec_llm(self, question: str, reponse: str, model: str = "gpt-4o-mini") -> dict:
        """
        Utilise un LLM comme juge pour évaluer une réponse.
        """
        prompt = f"""Tu es un évaluateur expert. Évalue la réponse suivante selon les critères donnés.

        QUESTION: {question}

        RÉPONSE À ÉVALUER: {reponse}

        CRITÈRES D'ÉVALUATION:
        {chr(10).join([f'- {k}: {v}' for k, v in self.criteres.items()])}

        Pour chaque critère, donne une note de 1 à 5 et une justification brève.
        Termine par une note globale et un résumé.

        Format de réponse (JSON):
        {{
            "scores": {{
                "pertinence": {{"note": X, "justification": "..."}},
                "exactitude": {{"note": X, "justification": "..."}},
                "completude": {{"note": X, "justification": "..."}},
                "clarte": {{"note": X, "justification": "..."}},
                "concision": {{"note": X, "justification": "..."}}
            }},
            "note_globale": X.X,
            "resume": "..."
        }}"""

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )

        return response.choices[0].message.content

# ============================================
# 7.2 COMPARAISON A/B
# ============================================

class ComparateurAB:
    """
    Compare deux variantes de prompts ou de modèles.
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model

    def comparer(self, question: str, reponse_a: str, reponse_b: str) -> dict:
        """
        Compare deux réponses et détermine laquelle est meilleure.
        """
        prompt = f"""Tu es un évaluateur impartial. Compare les deux réponses suivantes à la même question.

        QUESTION: {question}

        RÉPONSE A:
        {reponse_a}

        RÉPONSE B:
        {reponse_b}

        Analyse selon ces critères:
        1. Pertinence par rapport à la question
        2. Exactitude des informations
        3. Clarté et lisibilité
        4. Utilité pratique

        Donne ton verdict:
        - Quelle réponse est meilleure ?
        - Pourquoi ?
        - Y a-t-il des éléments à combiner des deux réponses ?

        Format JSON:
        {{
            "gagnant": "A" ou "B" ou "égalité",
            "raison": "...",
            "score_a": X/10,
            "score_b": X/10,
            "suggestions": "..."
        }}"""

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )

        return response.choices[0].message.content

# ============================================
# 7.3 ÉVALUATION AVEC DEEPEVAL
# ============================================

def installer_deepeval():
    """Instructions pour installer DeepEval."""
    print("""
    Pour utiliser DeepEval, installez-le avec:
    pip install deepeval

    Puis configurez votre clé OpenAI:
    export OPENAI_API_KEY=sk-...
    """)

# Exemple d'utilisation de DeepEval (à décommenter après installation)
"""
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase

# Créer un cas de test
test_case = LLMTestCase(
    input="Qu'est-ce que le machine learning ?",
    actual_output="Le machine learning est une branche de l'IA...",
    expected_output="Le machine learning est une technique d'apprentissage automatique...",
    context=["Le ML est une sous-branche de l'intelligence artificielle."]
)

# Définir les métriques
relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
faithfulness_metric = FaithfulnessMetric(threshold=0.7)

# Évaluer
evaluate([test_case], [relevancy_metric, faithfulness_metric])
"""

'\nfrom deepeval import evaluate\nfrom deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric\nfrom deepeval.test_case import LLMTestCase\n\n# Créer un cas de test\ntest_case = LLMTestCase(\n    input="Qu\'est-ce que le machine learning ?",\n    actual_output="Le machine learning est une branche de l\'IA...",\n    expected_output="Le machine learning est une technique d\'apprentissage automatique...",\n    context=["Le ML est une sous-branche de l\'intelligence artificielle."]\n)\n\n# Définir les métriques\nrelevancy_metric = AnswerRelevancyMetric(threshold=0.7)\nfaithfulness_metric = FaithfulnessMetric(threshold=0.7)\n\n# Évaluer\nevaluate([test_case], [relevancy_metric, faithfulness_metric])\n'

In [32]:
print("="*70)
print("EXERCICE 7 : ÉVALUATION DES LLMS")
print("="*70)

# Exercice 7.1 : Évaluation d'une réponse
evaluateur = EvaluateurManuel()

question_test = "Quels sont les avantages et inconvénients du télétravail ?"
reponse_test = """
Le télétravail présente plusieurs avantages :
- Flexibilité des horaires
- Économie de temps de transport
- Meilleur équilibre vie professionnelle/personnelle

Les inconvénients incluent :
- Isolement social
- Difficulté à séparer vie pro et perso
- Risque de surmenage

En conclusion, le télétravail peut être bénéfique si bien encadré.
"""

print("\nÉvaluation de la réponse:")
print("-"*40)
evaluation = evaluateur.evaluer_avec_llm(question_test, reponse_test)
print(evaluation)

EXERCICE 7 : ÉVALUATION DES LLMS

Évaluation de la réponse:
----------------------------------------
{
    "scores": {
        "pertinence": {"note": 5, "justification": "La réponse aborde directement les avantages et inconvénients du télétravail, répondant ainsi parfaitement à la question posée."},
        "exactitude": {"note": 5, "justification": "Les informations fournies sur les avantages et inconvénients du télétravail sont correctes et reflètent des réalités généralement reconnues."},
        "completude": {"note": 4, "justification": "Bien que la réponse couvre les principaux avantages et inconvénients, elle pourrait être enrichie par des exemples ou des détails supplémentaires."},
        "clarte": {"note": 5, "justification": "La réponse est bien structurée, avec une séparation claire entre les avantages et les inconvénients, ce qui facilite la compréhension."},
        "concision": {"note": 5, "justification": "La réponse est concise et va droit au but, sans informations sup

In [39]:
# Exercice 7.2 : Comparaison A/B de prompts
print("\n\nComparaison A/B:")
print("-"*40)

comparateur = ComparateurAB()

# Deux versions de réponses générées avec différents prompts
reponse_prompt_simple = """Le télétravail c'est bien car on ne perd pas de temps dans les transports et on peut travailler en pyjama. Mais c'est aussi nul car on ne voit plus personne."""

reponse_prompt_structure = """
## Avantages du télétravail

1. **Gain de temps** : Élimination des trajets domicile-travail
2. **Flexibilité** : Meilleure gestion de son emploi du temps
3. **Productivité** : Environnement souvent plus calme

## Inconvénients du télétravail

1. **Isolement** : Réduction des interactions sociales
2. **Frontières floues** : Difficulté à "débrancher"
3. **Communication** : Collaboration parfois plus complexe

## Recommandations

Un modèle hybride (2-3 jours en présentiel) semble être le meilleur compromis.
"""

resultat_comparaison = comparateur.comparer(
    question_test,
    reponse_prompt_simple,
    reponse_prompt_structure
)
print(resultat_comparaison)



Comparaison A/B:
----------------------------------------
```json
{
    "gagnant": "B",
    "raison": "La réponse B est plus structurée, détaillée et couvre à la fois les avantages et les inconvénients du télétravail de manière équilibrée. Elle fournit des informations précises et des recommandations pratiques, ce qui la rend plus utile.",
    "score_a": 4/10,
    "score_b": 9/10,
    "suggestions": "La réponse A pourrait être améliorée en ajoutant des détails et en structurant les idées de manière plus claire, tout en conservant un ton léger. Des éléments de la réponse A, comme le travail en pyjama, pourraient être intégrés dans la réponse B pour ajouter une touche personnelle."
}
```
